# **Shopee Code League - Order Brushing**

This notebook is for Shopee Code League Competition, Order Brushing, to detecting abnormal user behaviour. The competition details can be obtained from [here](https://www.kaggle.com/c/order-brushing-shopee-code-league/overview).

# **Import libraries and read dataset**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from datetime import datetime
from datetime import timedelta
from google.colab import files

In [2]:
from tensorflow.python.client import device_lib
check_device = device_lib.list_local_devices()
check_device

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18429616035528983126, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6292988458380020427
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16812768066705319756
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11102823701347151313
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
# order_brush_order.csv
d = pd.read_csv('order_brush_order.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
len(d.shopid.unique())

18770

In [4]:
# order_brush_order.csv
data = pd.read_csv('test_brush.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
data

,orderid,shopid,userid,event_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45
6,31388864143842,91242967,10209247,2019-12-30 15:07:45
7,31388868340962,68986766,65129646,2019-12-30 15:07:48
8,31388869521738,68986766,65129646,2019-12-30 15:07:49
9,31388867881658,68986766,86829555,2019-12-31 18:07:49


In [5]:
date = data['event_time']
date_time = []
for i in date:
  date_time.append(datetime.strptime(i,"%Y-%m-%d %H:%M:%S"))

In [6]:
data['date_time'] = date_time
data

,orderid,shopid,userid,event_time,date_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
6,31388864143842,91242967,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45
7,31388868340962,68986766,65129646,2019-12-30 15:07:48,2019-12-30 15:07:48
8,31388869521738,68986766,65129646,2019-12-30 15:07:49,2019-12-30 15:07:49
9,31388867881658,68986766,86829555,2019-12-31 18:07:49,2019-12-31 18:07:49


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   orderid     11 non-null     int64         
 1   shopid      11 non-null     int64         
 2   userid      11 non-null     int64         
 3   event_time  11 non-null     object        
 4   date_time   11 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 568.0+ bytes


In [8]:
b = data['date_time'].loc[10]
a = data['date_time'].loc[1]
delt = b-a
delt.total_seconds()

99791.0

In [9]:
data.loc[0:5]

,orderid,shopid,userid,event_time,date_time
0,31388857391599,91242967,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37
1,31388858606904,85669731,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,85669731,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,85669731,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
4,31388864143841,68986766,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
5,31388864143840,85669731,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45


In [10]:
check = data.loc[0:5].groupby('shopid')
check.first()

,orderid,userid,event_time,date_time
shopid,,,,
68986766,31388864143841,65129646,2019-12-30 15:07:45,2019-12-30 15:07:45
85669731,31388858606904,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
91242967,31388857391599,112494421,2019-12-30 15:07:37,2019-12-30 15:07:37


In [56]:
what_user = []
lol = pd.DataFrame(data.loc[0:5].groupby('userid')['orderid'].count().reset_index(name='user_count'))
lol

,userid,user_count
0,10209247,1
1,65129646,1
2,112494421,4


In [62]:
haha = lol[lol['user_count'] >= 3]
haha

,userid,user_count
2,112494421,4


In [68]:
tenten = list(haha['userid'])

In [69]:
tenten

[112494421]

In [61]:
key = list(check.groups.keys())
key

[68986766, 85669731, 91242967]

In [15]:
key[1]

85669731

In [16]:
len(key)

3

In [57]:
for k in key:
  print(k)

68986766
85669731
91242967


In [58]:
check_order = check.get_group(key[1])
check_order

,orderid,userid,event_time,date_time
1,31388858606904,112494421,2019-12-30 15:07:38,2019-12-30 15:07:38
2,31388860144322,112494421,2019-12-30 15:07:41,2019-12-30 15:07:41
3,31388862051153,112494421,2019-12-30 15:07:42,2019-12-30 15:07:42
5,31388864143840,10209247,2019-12-30 15:07:45,2019-12-30 15:07:45


In [18]:
check_order = check.get_group(key[1])
order = len(check_order.orderid)
buyers = len(check_order.userid.unique())
concentrate_rate = order/buyers
print(concentrate_rate)

2.0


In [19]:
charc = 'x'
haha = 'abc'
a_dict = {}
if charc in a_dict:
  a_dict[charc].append(haha)
else:
  a_dict[charc] = [haha]
print(a_dict)

{'x': ['abc']}


In [85]:
ans_dic= {}

for i in data.index:
  initial = data['date_time'].loc[i]
  for j in range(i, data.last_valid_index()+1):
    delta = data['date_time'].loc[j] - initial
    if(delta.total_seconds()<3600):
      check = data.loc[i:j].groupby('shopid')
      check['userid'].count()
      user_num = pd.DataFrame(data.loc[i:j].groupby('userid')['userid'].count().reset_index(name='user_count'))
      check_user = user_num[user_num['user_count'] >= 3]
      suspect_user = list(check_user['userid'])
      key = list(check.groups.keys())
      for k in key:
        check_order = check.get_group(k)
        order = len(check_order.orderid)
        buyers = len(check_order.userid.unique())
        concentrate_rate = order/buyers
        if(concentrate_rate >= 3):
          if k in ans_dic:
            ans_dic[k].append(suspect_user)
          else:
            ans_dic[k] = suspect_user

In [88]:
column_names = ["shopid", "userid"]
df = pd.DataFrame(columns = column_names)
df

,shopid,userid


In [86]:
for key, value in ans_dic.items():
  df['shopid'] key
  print(value)

85669731
[112494421, [112494421], [112494421], [112494421]]
68986766
[65129646, 112494421, [65129646, 112494421], [65129646], [65129646], [65129646]]


In [ ]:
column_names = ["shopid", "userid"]
df = pd.DataFrame(columns = column_names)
df

In [ ]:
df.to_csv('CY_Team_submission4.csv') 
files.download('CY_Team_submission4.csv')